In [1]:
import numpy as np
import pprint
from operator import itemgetter
from mdp_matrix import GridWorld

In [92]:
test_rewards = [[i, j, -1.0] for i in range(5) for j in range(5)]
# test_rewards[2] = [0, 2, 1]
test_rewards[23] = [4, 3, 1]
# test_rewards = [[0, 3, 5],
#                 [0, 1, 10]]
terminal_states = [23]
print test_rewards
gw = GridWorld(5, test_rewards, terminal_states)

[[0, 0, -1.0], [0, 1, -1.0], [0, 2, -1.0], [0, 3, -1.0], [0, 4, -1.0], [1, 0, -1.0], [1, 1, -1.0], [1, 2, -1.0], [1, 3, -1.0], [1, 4, -1.0], [2, 0, -1.0], [2, 1, -1.0], [2, 2, -1.0], [2, 3, -1.0], [2, 4, -1.0], [3, 0, -1.0], [3, 1, -1.0], [3, 2, -1.0], [3, 3, -1.0], [3, 4, -1.0], [4, 0, -1.0], [4, 1, -1.0], [4, 2, -1.0], [4, 3, 1], [4, 4, -1.0]]


In [93]:
print gw.T[0, 1, :]

[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


In [94]:
def sarsa(mdp, max_episode, alpha = 0.1, gamma = 0.9):
    # Initialization
    Q = [[0 for i in range(mdp.A)] for j in range(mdp.S)]
    old_Q = Q
    n_episode = 0
    epsilon = 0.1
    total_reward = 0
    while n_episode < max_episode: # TODO: Pick a finish condition for episode
        s = 0 # Initialize s, starting state
        
        # With prob epsilon, pick a random action
        if np.random.random_sample() <= epsilon:
            a = np.random.random_integers(0, mdp.A-1)         
        else:
            a = np.argmax(Q[s][:])
        while not mdp.is_terminal(s): # TODO: Finish episode/trajectory on terminal state
            # Observe S and R
            s_new = np.argmax(mdp.T[s, a, :]) # TODO: Change to stochastic ?
            r = mdp.R[s_new]
            T_new = np.zeros((mdp.S, mdp.S))
            
            # Pick new action A' from S'
            if np.random.random_sample() <= epsilon:
                a_new = np.random.random_integers(0, mdp.A-1)         
            else:
                a_new = np.argmax(Q[s_new][:])
            Q[s][a] = Q[s][a] + alpha*(r + gamma*Q[s_new][a_new] - Q[s][a])
            s = s_new
            a = a_new
            total_reward += r
        
        n_episode += 1
    return Q, total_reward/max_episode

In [98]:
[sarsa_Q, sarsa_avg_reward] = sarsa(gw, 100000)
print sarsa_avg_reward

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:24: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead


-5.74369


In [54]:
print np.reshape(np.argmax(sarsa_Q, 1), (5,5))

[[1 1 0 1 0]
 [1 1 2 3 2]
 [0 2 2 2 2]
 [1 0 1 0 0]
 [1 1 1 0 3]]


In [55]:
print np.reshape(gw.R, (5,5))

[[-1. -1.  1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1.  1. -1.]]


In [99]:
def expected_sarsa(mdp, max_episode, alpha = 0.1, gamma = 0.9):
    # Initialization
    Q = [[0.0 for i in range(mdp.A)] for j in range(mdp.S)]
    old_Q = Q
    n_episode = 0
    epsilon = 0.1
    total_reward = 0
    while n_episode < max_episode: # TODO: Pick a finish condition for episode
        s = 0 # Initialize s, starting state
        
        # With prob epsilon, pick a random action
        if np.random.random_sample() <= epsilon:
            a = np.random.random_integers(0, mdp.A-1)         
        else:
            a = np.argmax(Q[s][:])
        r = 0
        while not mdp.is_terminal(s): # TODO: Finish episode/trajectory on terminal state
            # Observe S and R
            s_new = np.argmax(mdp.T[s, a, :]) # TODO: Change to stochastic ?
            r = mdp.R[s_new]
            T_new = np.zeros((mdp.S, mdp.S))
            
            # Pick new action A' from S'
            if np.random.random_sample() <= epsilon:
                a_new = np.random.random_integers(0, mdp.A-1)         
            else:
                a_new = np.argmax(Q[s_new][:])
                
            best_action = np.argmax(Q[s_new][:])
            Q[s][a] = Q[s][a] + alpha*(r + gamma*((1-epsilon)*Q[s_new][best_action]+(epsilon/mdp.A)*sum(Q[s_new][act] for act in range(mdp.A))) - Q[s][a])

            s = s_new
            a = a_new
            total_reward += r
        
        n_episode += 1
    return Q, total_reward/max_episode

In [100]:
Q = [[1 for i in range(gw.A)] for j in range(gw.S)]
sum(Q[0][act] for act in range(gw.A))
# print Q[0]

4

In [97]:
[expected_sarsa_Q, expected_sarsa_avg]  = expected_sarsa(gw, 100000)
print np.reshape(np.argmax(expected_sarsa_Q, 1), (5,5))
print expected_sarsa_avg

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:25: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:13: DeprecationWarning: This function is deprecated. Please call randint(0, 3 + 1) instead


[[0 1 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [1 1 0 0 3]
 [1 1 1 0 3]]
-5.67459


In [18]:
print Q

[[-2.0768882920173954, -0.27275823905339325, -1.3269005750268998, -1.4113307873599377], [-1.1638077283033614, 0.99999999999999956, -0.13459129625773833, -1.5396825824699591], [0, 0, 0, 0], [-0.10000000000000001, 0, 0, 0], [-0.10000000000000001, 0, 0, 0], [-1.3188744910209573, -1.1386351932581618, -1.403534225840114, -1.3673190107215827], [-0.81450172174137792, -0.10444261893667516, -0.42204225343850588, -0.77852532042991984], [-0.66498745958849459, -0.41800832761019591, 0.99999979135577033, -0.24258152385048889], [-0.10000000000000001, -0.10000000000000001, -0.10000000000000001, -0.047060335515935789], [-0.10000000000000001, -0.19, -0.10000000000000001, -0.10000000000000001], [-0.96608485687594126, -0.94727610741009238, -0.94425558071387883, -1.0224273179147627], [-0.69963670000000011, -0.65529542623167436, -0.6291015200381348, -0.68418227239199003], [-0.36544690000000002, -0.35209000000000001, -0.28733865818061, -0.36316178233508378], [-0.150949, -0.10000000000000001, -0.1015461081889